# Host Tools

Command for running tool server:\
`python host_local_tools.py`

# Custom Parser

In [16]:
import importlib
import json
import os
import requests
import yaml
from typing import Any, Dict, List, Union, NamedTuple
import sys
import re

from bmtools.agent.apitool import RequestTool
from bmtools.agent.executor import Executor, AgentExecutorWithTranslation
from bmtools import get_logger
from bmtools.agent.BabyagiTools import BabyAGI
from bmtools.agent.singletool import import_all_apis, load_single_tools
from bmtools import get_logger

from langchain.llms import OpenAI
from langchain import OpenAI, LLMChain, PromptTemplate, SerpAPIWrapper
from langchain.agents import ZeroShotAgent, AgentExecutor, initialize_agent, Tool
from langchain.agents.agent import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS

sys.path.insert(0, 'D:/Coding/ToolBench/data-generation')
from answer.utils import prepare_queries, NAME2URL, MyZeroShotAgent, MyAgentExecutor, MyMRKLOutputParser, LogParser

In [13]:
# basically copy the codes in singletool.py
class STQuestionAnswerer:
    def __init__(self, stream_output=False, llm_model=None):
        self.llm = llm_model
        self.stream_output = stream_output

    def load_tools(self, name, meta_info, prompt_type="react-with-tool-description", return_intermediate_steps=True):
        self.all_tools_map = {}
        self.all_tools_map[name] = import_all_apis(meta_info)

        logger.info("Tool [{}] has the following apis: {}".format(name, self.all_tools_map[name]))
        tool_str = "; ".join([t.name for t in self.all_tools_map[name]])

        prefix = f"""Answer the following questions as best you can. Specifically, you have access to the following APIs:"""
        suffix = """Begin! Remember: (1) Follow the format, i.e,\nThought:\nAction:\nAction Input:\nObservation:\nFinal Answer:\n (2) Provide as much as useful information in your Final Answer. (3) Do not make up anything, and if your Observation has no link, DO NOT hallucihate one. (4) If you have enough information and want to stop the process, please use \nThought: I have got enough information\nFinal Answer: **your response in simplified chinese. \n The Action: MUST be one of the following:""" + tool_str + """\nQuestion: {input}\n Agent scratchpad (history actions):\n {agent_scratchpad}"""

        # Create prompt
        prompt = ZeroShotAgent.create_prompt(
            self.all_tools_map[name],
            prefix=prefix,
            suffix=suffix,
#             prefix=CPREFIX,
#             format_instructions=CFORMAT_INSTRUCTIONS,
#             suffix=CSUFFIX,
            input_variables=["input", "agent_scratchpad"]
        )
        
        print(prompt)

        llm_chain = LLMChain(
            llm=self.llm,
            prompt=prompt
        )
        logger.info("Full prompt template: {}".format(prompt.template))
        tool_names = [tool.name for tool in self.all_tools_map[name] ]
        agent = MyZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
        agent.output_parser = MyMRKLOutputParser()
        if self.stream_output:
            agent_executor = Executor.from_agent_and_tools(agent=agent, tools=self.all_tools_map[name] , verbose=True, return_intermediate_steps=return_intermediate_steps)
        else:
            agent_executor = MyAgentExecutor.from_agent_and_tools(agent=agent, tools=self.all_tools_map[name], verbose=True, return_intermediate_steps=return_intermediate_steps)
        
        agent_executor.early_stopping_method = "generate"
        # print(type(prompt.template))
        print(prompt.template)
        return agent_executor, prompt.template

In [3]:
# class MyZeroShotAgent(ZeroShotAgent):
#     @property   
#     def return_values(self) -> List[str]:
#         """Return values of the agent."""
#         return ["output", "log"]

#     @property
#     def llm_prefix(self) -> str:
#         """Prefix to append the llm call with."""
#         return ""

# Single Tool

In [4]:
from bmtools.agent.singletool import load_single_tools
from my_utils.load_keys import load_key
import langchain
import os

In [5]:
load_key("openai")

Loaded openai API


In [14]:
# Langchain
logger = get_logger(__name__)
tool_name, tool_url = 'weather',  "http://127.0.0.1:8079/tools/weather/"
tool_name, tool_config = load_single_tools(tool_name, tool_url)
print(tool_name, tool_config)
stqa =  STQuestionAnswerer(llm_model=langchain.OpenAI(temperature=0, max_tokens=512))

agent, prompt_template = stqa.load_tools(tool_name, tool_config, prompt_type="react-with-tool-description")
agent.return_intermediate_steps = True
agent("write a weather report for SF today")

# """ 
# from bmtools.agent.singletool import load_single_tools, STQuestionAnswerer

# tool_name, tool_url = 'wikipedia',  "http://127.0.0.1:8079/tools/wikipedia/"
# tool_name, tool_config = load_single_tools(tool_name, tool_url)
# print(tool_name, tool_config)
# stqa =  STQuestionAnswerer()

# agent = stqa.load_tools(tool_name, tool_config, prompt_type="babyagi")
# # agent = stqa.load_tools(tool_name, tool_config, prompt_type="react-with-tool-description")# prompt_type="babyagi")
# agent("Where is Yaoming Born?")
# """

[INFO|(BMTools)singletool:25]2023-07-12 14:20:52,119 >>  Doc string URL: http://127.0.0.1:8079/tools/weather/openapi.json
[INFO|(BMTools)singletool:34]2023-07-12 14:20:52,119 >>  server_url http://127.0.0.1:8079/tools/weather
[INFO|(BMTools)apitool:146]2023-07-12 14:20:52,119 >>  API Name: get_weather_today
[INFO|(BMTools)apitool:147]2023-07-12 14:20:52,119 >>  API Description: Get today's the weather. Your input should be a json (args json schema): {{"location" : string, }} The Action to trigger this API should be get_weather_today and the input parameters should be a json dict string. Pay attention to the type of parameters.
[INFO|(BMTools)apitool:146]2023-07-12 14:20:52,119 >>  API Name: forecast_weather
[INFO|(BMTools)apitool:147]2023-07-12 14:20:52,119 >>  API Description: Forecast weather in the upcoming days.. Your input should be a json (args json schema): {{"location" : string, "days" : integer, }} The Action to trigger this API should be forecast_weather and the input paramet

weather {'schema_version': 'v1', 'name_for_human': 'Weather Info', 'name_for_model': 'Weather', 'description_for_human': 'Look up weather information', 'description_for_model': 'Plugin for look up weather information', 'auth': {'type': 'none'}, 'api': {'type': 'openapi', 'url': 'http://127.0.0.1:8079/tools/weather/openapi.json', 'is_user_authenticated': False}, 'author_github': None, 'logo_url': 'https://cdn.weatherapi.com/v4/images/weatherapi_logo.png', 'contact_email': 'hello@contact.com', 'legal_info_url': 'hello@legal.com'}
input_variables=['input', 'agent_scratchpad'] output_parser=None partial_variables={} template='Answer the following questions as best you can. Specifically, you have access to the following APIs:\n\nget_weather_today: Get today\'s the weather. Your input should be a json (args json schema): {{"location" : string, }} The Action to trigger this API should be get_weather_today and the input parameters should be a json dict string. Pay attention to the type of para

{'input': 'write a weather report for SF today',
 'output': '今天旧金山的天气报告：总体情况为多云，名称为旧金山，地区为加利福尼亚州，国家为美国，当地时间为2023年7月11日23时20分，温度为13.3摄氏度（55.9华氏度），降水量为0.0毫米（0.0英寸），气压为1016.0米比弗，湿度为90，云量为50，体感温度为12.2摄氏度（53.9华氏度），风速为19.4公里/小时（12.1英里/小时），能见度为16.0公里（9.0英里），紫外线指数为1.0。',
 'intermediate_steps': [(AgentAction(tool='get_weather_today', tool_input='{"location": "SF"}', log='\nThought: I need to get the weather for SF today\nAction: get_weather_today\nAction Input: {"location": "SF"}'),
   '"Today\'s weather report for San Francisco is:\\noverall: Partly cloudy,\\nname: San Francisco,\\nregion: California,\\ncountry: United States of America,\\nlocaltime: 2023-07-11 23:20,\\ntemperature: 13.3(C), 55.9(F),\\npercipitation: 0.0(mm), 0.0(inch),\\npressure: 1016.0(milibar),\\nhumidity: 90,\\ncloud: 50,\\nbody temperature: 12.2(C), 53.9(F),\\nwind speed: 19.4(kph), 12.1(mph),\\nvisibility: 16.0(km), 9.0(miles),\\nUV index: 1.0,\\n"')],
 'log': 'Thought: I now know the final answer\nFinal Answer: 今天旧金山的天

# Multi Tool

In [9]:
from bmtools.agent.tools_controller import load_valid_tools, MTQuestionAnswerer
from my_utils.load_keys import load_key


## Load API Keys

In [10]:
load_key("openai")

Loaded openai API


In [11]:
tools_mappings = {
    'weather': "http://127.0.0.1:8079/tools/weather/",
    'wolframalpha': "http://127.0.0.1:8079/tools/wolframalpha/",
    'notool': "http://127.0.0.1:8079/tools/notool/"
}

tools = load_valid_tools(tools_mappings)

In [17]:
qa = MTQuestionAnswerer(openai_api_key='', all_tools=tools)

agent = qa.build_runner()

agent("How are you")

[INFO|(BMTools)tools_controller:52]2023-07-12 14:24:40,026 >>  Using ChatGPT
[INFO|(BMTools)tools_controller:61]2023-07-12 14:24:40,028 >>  All tools: {'weather': {'schema_version': 'v1', 'name_for_human': 'Weather Info', 'name_for_model': 'Weather', 'description_for_human': 'Look up weather information', 'description_for_model': 'Plugin for look up weather information', 'auth': {'type': 'none'}, 'api': {'type': 'openapi', 'url': 'http://127.0.0.1:8079/tools/weather/openapi.json', 'is_user_authenticated': False}, 'author_github': None, 'logo_url': 'https://cdn.weatherapi.com/v4/images/weatherapi_logo.png', 'contact_email': 'hello@contact.com', 'legal_info_url': 'hello@legal.com'}, 'wolframalpha': {'schema_version': 'v1', 'name_for_human': 'Wolfram', 'name_for_model': 'Wolfram', 'description_for_human': 'Access computation, math, curated knowledge & real-time data through Wolfram|Alpha and Wolfram Language.', 'description_for_model': '"Dynamic computation and curated data from WolframAl

[INFO|(BMTools)singletool:114]2023-07-12 14:24:40,035 >>  Full prompt template: Answer the following questions as best you can. General instructions are: "Dynamic computation and curated data from WolframAlpha and Wolfram Cloud.
Only use the getWolframAlphaResults endpoints; all other Wolfram endpoints are deprecated.
Prefer getWolframAlphaResults unless Wolfram Language code should be evaluated.
Try to include images returned by getWolframAlphaResults. Queries to getWolframAlphaResults must ALWAYS have this structure: {{"input": query}}.
",. Specifically, you have access to the following APIs:

getWolframAlphaResults: Get Wolfram|Alpha results using natural query. Queries to getWolframAlphaResults must ALWAYS have this structure: {{"input": query}}. And please directly read the output json.. Your input should be a json (args json schema): {{"input" : string, }} The Action to trigger this API should be getWolframAlphaResults and the input parameters should be a json dict string. Pay at

{
    "thoughts": {
        "text": "I should start by checking the weather.",
        "reasoning": "Knowing the weather can help me plan my next actions."
    },
    "command": {
        "name": "Weather",
        "args": {
            "goal": "Check the current weather"
        }
    }
}


AttributeError: 'AgentFinish' object has no attribute 'name'

# LangChain

## Load API Keys

In [ ]:
load_key("serpapi")

## Example

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

In [ ]:
# The language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
# Let's test it out!
agent.run("Tell me about yourself")